Explorar los datos mediante consultas SQL y responder las preguntas planteadas sobre calidad y transformación de los datos:

¿Cuáles fueron los 5 productos más vendidos (por cantidad total), y cuál fue el vendedor que más unidades vendió de cada uno? Una vez obtenga los resultados, en el análisis responde: ¿Hay algún vendedor que aparece más de una vez como el que más vendió un producto? ¿Algunos de estos vendedores representan más del 10% de la ventas de este producto?

Entre los 5 productos más vendidos, ¿cuántos clientes únicos compraron cada uno y qué proporción representa sobre el total de clientes? Analiza si ese porcentaje sugiere que el producto fue ampliamente adoptado entre los clientes o si, por el contrario, fue comprado por un grupo reducido que generó un volumen alto de ventas. Compara los porcentajes entre productos e identifica si alguno de ellos depende más de un segmento específico de clientes

In [47]:

import utils.sql_utils as sql_utils
import utils.notebook_utils as notebook_utils
query = """
SELECT 
    p.productID AS product_id,
    p.productName AS product_name,
    SUM(s.quantity) AS total_quantity
FROM sales s
JOIN products p ON s.productID = p.productID
GROUP BY p.productID, p.productName
ORDER BY total_quantity DESC
LIMIT 5;
"""

notebook_utils.print_colored('TOP 5 products', 'green')
sql_utils.run_query(query=query)


,product_id,product_name,total_quantity
0,179,Yoghurt Tubes,199724.0
1,161,Longos - Chicken Wings,199659.0
2,47,Thyme - Lemon; Fresh,198567.0
3,280,Onion Powder,198163.0
4,103,Cream Of Tartar,198126.0


In [ ]:
import utils.sql_utils as sql_utils

query= """
WITH top_products AS (
    SELECT 
        p.productID AS product_id,
        p.productName AS product_name,
        SUM(s.quantity) AS total_quantity
    FROM products p
    JOIN sales s ON p.productID = s.productID
    GROUP BY p.productID, p.productName
    ORDER BY total_quantity DESC
    LIMIT 5
),
seller_sales AS (
    SELECT 
        tp.product_id,
        tp.product_name,
        tp.total_quantity AS total_sold,
        s.salesPersonID AS seller_id,
        SUM(s.quantity) AS seller_quantity
    FROM top_products tp
    JOIN sales s ON tp.product_id = s.productID
    GROUP BY tp.product_id, tp.product_name, s.salesPersonID
),
ranked_sellers AS (
    SELECT 
        *,
        RANK() OVER (PARTITION BY product_id ORDER BY seller_quantity DESC) AS seller_rank
    FROM seller_sales
)
SELECT 
    rs.product_id,
    rs.product_name,
    rs.seller_id,
    rs.total_sold,
    CONCAT( e.FirstName ,' ', e.LastName) AS seller_name,
    rs.seller_quantity
FROM ranked_sellers rs
JOIN employees e ON rs.seller_id = e.employeeID
WHERE seller_rank = 1
ORDER BY seller_quantity DESC;
"""
# SELECT 
#     productID,
#     productName,
#     sellerID,
#     seller_quantity,
#     total_sold
# FROM ranked_sellers
# WHERE seller_rank = 1;
sql_utils.run_query(query=query)

,product_id,product_name,seller_id,total_sold,seller_name,seller_quantity
0,179,Yoghurt Tubes,9,199724.0,Daphne King,10285.0
1,161,Longos - Chicken Wings,10,199659.0,Jean Vang,10785.0
2,103,Cream Of Tartar,11,198126.0,Sonya Dickson,10348.0
3,47,Thyme - Lemon; Fresh,21,198567.0,Devon Brewer,11050.0
4,280,Onion Powder,21,198163.0,Devon Brewer,10570.0


In [50]:
# Entre los 5 productos más vendidos, ¿cuántos clientes únicos compraron cada uno y qué proporción representa sobre el total de clientes?

query = """
WITH top_products AS (
    SELECT 
        p.productID AS product_id,  
        p.productName AS product_name,
        SUM(s.quantity) AS total_quantity
    FROM products p
    JOIN sales s ON p.productID = s.productID
    GROUP BY p.productID, p.productName
    ORDER BY total_quantity DESC
    LIMIT 5
),
customer_counts AS (
    SELECT
        tp.product_id,
        COUNT(DISTINCT s.customerID) AS unique_customers
    FROM top_products tp
    JOIN sales s ON tp.product_id = s.productID
    GROUP BY tp.product_id
),
total_customers AS (
    SELECT COUNT(DISTINCT customerID) AS total_customers
    FROM sales
)
SELECT
    cc.product_id,
    tp.product_name,
    cc.unique_customers,
    tc.total_customers,
    ROUND((cc.unique_customers/ tc.total_customers) * 100, 2) AS proportion_percentage
FROM customer_counts cc
JOIN top_products tp ON cc.product_id = tp.product_id
JOIN total_customers tc ON true
ORDER BY cc.unique_customers DESC;
"""
notebook_utils.print_colored('TOP 5 products with unique customers', 'green')
sql_utils.run_query(query=query)

,product_id,product_name,unique_customers,total_customers,proportion_percentage
0,161,Longos - Chicken Wings,14252,98759,14.43
1,103,Cream Of Tartar,14246,98759,14.43
2,47,Thyme - Lemon; Fresh,14101,98759,14.28
3,179,Yoghurt Tubes,14066,98759,14.24
4,280,Onion Powder,14058,98759,14.23


A qué categorías pertenecen los 5 productos más vendidos y qué proporción representan dentro del total de unidades vendidas de su categoría? Utiliza funciones de ventana para comparar la relevancia de cada producto dentro de su propia categoría.

¿Cuáles son los 10 productos con mayor cantidad de unidades vendidas en todo el catálogo y cuál es su posición dentro de su propia categoría? Utiliza funciones de ventana para identificar el ranking de cada producto en su categoría. Luego, analiza si estos productos son también los líderes dentro de sus categorías o si compiten estrechamente con otros productos de alto rendimiento. ¿Qué observas sobre la concentración de ventas dentro de algunas categorías?

In [61]:

import utils.sql_utils as sql_utils
import utils.notebook_utils as notebook_utils

query = """
WITH top_sales AS (
    SELECT 
        p.productID AS product_id,
        p.productName AS product_name,
        p.categoryID AS category_id,
        c.categoryName AS category_name,
        SUM(s.quantity) AS total_quantity
    FROM sales s
    JOIN products p ON s.productID = p.productID
    JOIN categories c ON p.categoryID = c.categoryID
    GROUP BY p.productID, p.productName, p.categoryID, c.categoryName
    ORDER BY total_quantity DESC
    LIMIT 5
),
category_totals AS (
    SELECT 
        ts.category_id,
        SUM(s.quantity) AS total_category_quantity
    FROM sales s
    JOIN products p ON s.productID = p.productID
    JOIN top_sales ts ON ts.category_id = p.categoryID
    GROUP BY ts.category_id
)
SELECT 
    ts.product_id,
    ts.product_name,
    ts.category_name,
    ts.total_quantity,
    ct.total_category_quantity,
    ROUND(100.0 * ts.total_quantity / ct.total_category_quantity, 2) AS category_percentage
FROM top_sales ts
JOIN category_totals ct ON ts.category_id = ct.category_id
ORDER BY category_percentage DESC;
"""

notebook_utils.print_colored('TOP 5 products', 'green')
sql_utils.run_query(query=query)

,product_id,product_name,category_name,total_quantity,total_category_quantity,category_percentage
0,179,Yoghurt Tubes,Seafood,199724.0,6996142.0,2.85
1,161,Longos - Chicken Wings,Snails,199659.0,7199358.0,2.77
2,280,Onion Powder,Beverages,198163.0,7393693.0,2.68
3,47,Thyme - Lemon; Fresh,Poultry,198567.0,9159792.0,2.17
4,103,Cream Of Tartar,Meat,198126.0,9719274.0,2.04


In [53]:

import utils.sql_utils as sql_utils
import utils.notebook_utils as notebook_utils
query = """SELECT 
        c.categoryID AS category_id,
        SUM(s.quantity) AS total_category_sales
    FROM sales s
    JOIN products p ON s.productID = p.productID
    JOIN categories c ON p.categoryID = c.categoryID
    GROUP BY c.categoryID
    """

notebook_utils.print_colored('TOP 5 products', 'green')
sql_utils.run_query(query=query)

,category_id,total_category_sales
0,1,11078448.0
1,8,5433152.0
2,11,8368793.0
3,6,6996142.0
4,9,9159792.0
5,3,8735255.0
6,4,6815143.0
7,7,9719274.0
8,2,6983451.0
9,5,7393693.0


In [64]:

import utils.sql_utils as sql_utils
import utils.notebook_utils as notebook_utils

query = """WITH top_sales AS (
    SELECT 
        s.productID AS product_id,
        p.productName AS product_name,
        p.categoryID AS category_id,
        c.categoryName AS category_name,
        SUM(s.quantity) AS total_quantity
    FROM sales s
    JOIN products p ON s.productID = p.productID
    JOIN categories c ON p.categoryID = c.categoryID
    GROUP BY s.productID, p.productName, p.categoryID, c.categoryName
    ORDER BY total_quantity DESC
    LIMIT 5
),
category_totals AS (
    SELECT 
        p.categoryID AS category_id,
        SUM(s.quantity) AS total_category_quantity
    FROM sales s
    JOIN products p ON s.productID = p.productID
    WHERE p.categoryID IN (SELECT DISTINCT category_id FROM top_sales)
    GROUP BY p.categoryID
)
SELECT 
    ts.product_id,
    ts.product_name,
    ts.category_name,
    ts.total_quantity,
    ct.total_category_quantity,
    ROUND(100.0 * ts.total_quantity / ct.total_category_quantity, 2) AS category_percentage
FROM top_sales ts
JOIN category_totals ct ON ts.category_id = ct.category_id
ORDER BY category_percentage DESC;
    """

notebook_utils.print_colored('TOP 5 products', 'green')
sql_utils.run_query(query=query)

,product_id,product_name,category_name,total_quantity,total_category_quantity,category_percentage
0,179,Yoghurt Tubes,Seafood,199724.0,6996142.0,2.85
1,161,Longos - Chicken Wings,Snails,199659.0,7199358.0,2.77
2,280,Onion Powder,Beverages,198163.0,7393693.0,2.68
3,47,Thyme - Lemon; Fresh,Poultry,198567.0,9159792.0,2.17
4,103,Cream Of Tartar,Meat,198126.0,9719274.0,2.04


In [65]:
#¿Cuáles son los 10 productos con mayor cantidad de unidades vendidas en todo el catálogo y cuál es su posición dentro de su propia categoría? Utiliza funciones de ventana para identificar el ranking de cada producto en su categoría. Luego, analiza si estos productos son también los líderes dentro de sus categorías o si compiten estrechamente con otros productos de alto rendimiento. ¿Qué observas sobre la concentración de ventas dentro de algunas categorías?

query = """
WITH product_sales AS (
    SELECT 
        p.productID AS product_id,
        p.productName AS product_name,
        p.categoryID AS category_id,
        c.categoryName AS category_name,
        SUM(s.quantity) AS total_quantity
    FROM sales s
    JOIN products p ON s.productID = p.productID
    JOIN categories c ON p.categoryID = c.categoryID
    GROUP BY p.productID, p.productName, p.categoryID, c.categoryName
),
ranked_products AS (
    SELECT 
        ps.product_id,
        ps.product_name,
        ps.category_id,
        ps.category_name,
        ps.total_quantity,
        RANK() OVER (PARTITION BY ps.category_id ORDER BY ps.total_quantity DESC) AS category_rank
    FROM product_sales ps
)
SELECT 
    rp.product_id,
    rp.product_name,
    rp.category_name,
    rp.total_quantity,  
    rp.category_rank
FROM ranked_products rp
ORDER BY rp.total_quantity DESC
LIMIT 10;
"""
notebook_utils.print_colored('TOP 10 products with category ranking', 'green')
sql_utils.run_query(query=query)

,product_id,product_name,category_name,total_quantity,category_rank
0,179,Yoghurt Tubes,Seafood,199724.0,1
1,161,Longos - Chicken Wings,Snails,199659.0,1
2,47,Thyme - Lemon; Fresh,Poultry,198567.0,1
3,280,Onion Powder,Beverages,198163.0,1
4,103,Cream Of Tartar,Meat,198126.0,1
5,39,Dried Figs,Produce,198032.0,1
6,324,Apricots - Dried,Snails,198032.0,2
7,319,Towels - Paper / Kraft,Meat,198005.0,2
8,425,Wine - Redchard Merritt,Dairy,197969.0,1
9,184,Hersey Shakes,Poultry,197942.0,2
